# Data preparation

The aim of this notebook is to collect the information about the credible set lead variants.
This includes:
* Addition of Major population sample size and size of cases/controls from studyIndex, 
* Addition of most severe consequences and consequence score derived from VEP annotations from variantIndex
* Calculation of MAF (Minor Allele Frequency) for lead variants
* Calculation of Variance Explained by lead variant

### Data extraction and loading

Data for this analysis has to be downloaded from 3 datasets available by FTP:
* credible_set
* variant
* study

In [ ]:
!rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/25.03/output/credible_set .
!rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/25.03/output/study .
!rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/25.03/output/variant .


In [ ]:
# Ensure proper java version < 11
!java -version


openjdk version "11.0.13" 2021-10-19
OpenJDK Runtime Environment JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21)
OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)


#### Loading the data with gentropy


In [ ]:
from gentropy.common.session import Session
from gentropy.dataset.study_index import StudyIndex
from gentropy.dataset.study_locus import StudyLocus
from gentropy.dataset.variant_index import VariantIndex
from pyspark.sql import Column
from pyspark.sql import functions as f


In [ ]:
session = Session(extended_spark_conf={"spark.driver.memory": "40G"})
variant_index_path = "variant"
study_index_path = "study"
credible_set_path = "credible_set"


In [4]:
session.spark


In [6]:
vi = VariantIndex.from_parquet(session, variant_index_path)
si = StudyIndex.from_parquet(session, study_index_path)
cs = StudyLocus.from_parquet(session, credible_set_path)


In [ ]:
vi.df.show(n=1)
si.df.show(n=1)
cs.df.show(n=1)


+----------------+----------+---------+---------------+---------------+--------------------+-----------------------+----------------------+-----------+-----------------+--------------------+--------------------+--------------------+
|       variantId|chromosome| position|referenceAllele|alternateAllele|       variantEffect|mostSevereConsequenceId|transcriptConsequences|      rsIds|           hgvsId|   alleleFrequencies|             dbXrefs|  variantDescription|
+----------------+----------+---------+---------------+---------------+--------------------+-----------------------+----------------------+-----------+-----------------+--------------------+--------------------+--------------------+
|10_104021953_A_G|        10|104021953|              A|              G|[{VEP, intron_var...|             SO_0001627|  [{[SO_0001632], N...|[rs4918077]|10:g.104021953A>G|[{sas_adj, 0.4902...|[{rs4918077, ense...|Intron variant ov...|
+----------------+----------+---------+---------------+-------------

25/04/07 09:04:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------+---------+--------------------+------------------------+-------------+------+---------------------+-----------+--------+----------------+----------------------+---------------+------------------+----------------------------------+--------------------+--------------------+------+---------+--------+---------+---------------------+-------------------+------------------+---------------+-------------+--------------------+-----------+---------+---------------+
|             studyId|  projectId|studyType|     traitFromSource|traitFromSourceMappedIds|   diseaseIds|geneId|biosampleFromSourceId|biosampleId|pubmedId|publicationTitle|publicationFirstAuthor|publicationDate|publicationJournal|backgroundTraitFromSourceMappedIds|backgroundDiseaseIds|   initialSampleSize|nCases|nControls|nSamples|  cohorts|ldPopulationStructure|   discoverySamples|replicationSamples|qualityControls|analysisFlags|summarystatsLocation|hasSumstats|condition|sumstatQCValues|
+-------------

#### Build dataset for analysis
Code below collects all required fields required to perform analysis on MAF and variant effects

In [ ]:
def major_population_in_study(ld_col: Column, default_major_pop: str = "nfe") -> Column:
    """Extract the major population from the study ld population structure.

    Args:
        ld_col (Column): ld population structure field  array<struct<ldPopulation: string, relativeSampleSize: double>>
        default_major_pop (str, optional): population to use as default, when no population was reported. Defaults to "nfe".

    Returns:
        Column: ld_col struct

    """

    def reduce_pops(pop1: Column, pop2: Column) -> Column:
        """Reduce two populations based on relative sample size.

        This function takes 2 populations and report one of them based on following conditions:
        * Use pop with bigger relativeSampleSize
        * In case of a tie, the default_major_pop is preferred,
        * In case of tie and no default_major_pop in pop1 and pop2, use pop1.
        """
        return (
            f.when(
                pop1.getField("relativeSampleSize") > pop2.getField("relativeSampleSize"),
                pop1,
            )
            .when(
                pop1.getField("relativeSampleSize") < pop2.getField("relativeSampleSize"),
                pop2,
            )
            .when(
                (
                    (pop1.getField("relativeSampleSize") == pop2.getField("relativeSampleSize"))
                    & (pop1.getField("ldPopulation") == f.lit(default_major_pop))
                ),
                pop1,
            )
            .when(
                (
                    (pop1.getField("relativeSampleSize") == pop2.getField("relativeSampleSize"))
                    & (pop2.getField("ldPopulation") == f.lit(default_major_pop))
                ),
                pop2,
            )
            .otherwise(pop1)
        )

    fallback = f.struct(f.lit(default_major_pop).alias("ldPopulation"), f.lit(0.0).alias("relativeSampleSize"))

    return f.when(
        f.size(ld_col) > 0,
        f.reduce(
            ld_col,
            fallback,
            reduce_pops,
        ),
    ).otherwise(fallback)


def vep_variant_effect(c: Column) -> Column:
    """Extract VEP variant effect."""

    def extract_fields(ve: Column) -> Column:
        return f.struct(
            ve.getField("assessment").alias("assessment"),
            ve.getField("normalisedScore").alias("normalisedScore"),
            ve.getField("targetId").alias("targetId"),
        )

    return f.transform(f.filter(c, lambda ve: ve.getField("method") == f.lit("VEP")), extract_fields).getItem(0)


def major_population_allele_freq(major_pop: Column, allele_freq: Column) -> Column:
    """Extract major population from variant.alleleFrequencies."""
    return f.filter(
        allele_freq,
        lambda freq: f.replace(freq.getField("populationName"), f.lit("_adj"), f.lit(""))
        == major_pop.getField("ldPopulation"),
    )


def maf(variant_freq: Column) -> Column:
    """Calculate Minor Allele Frequency from variant frequency."""
    return (
        f.when(
            ((f.size(variant_freq) == 1) & (variant_freq.getItem(0).getField("alleleFrequency") > 0.5)),
            f.lit(1.0) - variant_freq.getItem(0).getField("alleleFrequency"),
        )
        .when(
            ((f.size(variant_freq) == 1) & (variant_freq.getItem(0).getField("alleleFrequency") <= 0.5)),
            variant_freq.getItem(0).getField("alleleFrequency"),
        )
        .when(f.size(variant_freq) == 0, f.lit(0.0))
    )


In [93]:
_cs = cs.df.select(
    f.col("studyId"),
    f.col("studyLocusId"),
    f.col("variantId"),
    f.col("beta"),
    f.col("zScore"),
    f.col("pValueMantissa"),
    f.col("pValueExponent"),
    f.col("standardError"),
    f.col("finemappingMethod"),
    f.col("studyType"),
    f.size("locus").alias("credibleSetSize"),
)
_si = si.df.select(
    f.col("studyId"),
    f.col("nSamples"),
    f.col("nControls"),
    f.col("nCases"),
    major_population_in_study(f.col("ldPopulationStructure"), "nfe").alias("majorPopulation"),
)

_vi = vi.df.select(
    f.col("variantId"),
    f.col("allelefrequencies"),
    vep_variant_effect("variantEffect").alias("vepEffect"),
)


In [ ]:
_si.groupBy("majorPopulation.ldPopulation").count().show()


+------------+-------+
|ldPopulation|  count|
+------------+-------+
|         fin|   2303|
|         afr|   6707|
|         nfe|1940532|
|         eas|  11904|
|         amr|    923|
+------------+-------+



In [ ]:
_vi.groupBy("vepEffect.assessment").count().orderBy(f.desc("count")).show(truncate=False, n=300)


+-----------------------------------+-------+
|assessment                         |count  |
+-----------------------------------+-------+
|intron_variant                     |3233318|
|upstream_gene_variant              |1409198|
|missense_variant                   |819351 |
|synonymous_variant                 |413178 |
|3_prime_UTR_variant                |126781 |
|frameshift_variant                 |100888 |
|non_coding_transcript_exon_variant |91173  |
|splice_region_variant              |74404  |
|splice_polypyrimidine_tract_variant|57500  |
|stop_gained                        |57247  |
|splice_donor_variant               |20958  |
|5_prime_UTR_variant                |20775  |
|splice_acceptor_variant            |17843  |
|splice_donor_region_variant        |15931  |
|inframe_deletion                   |14399  |
|splice_donor_5th_base_variant      |7486   |
|inframe_insertion                  |6072   |
|downstream_gene_variant            |2801   |
|start_lost                       

In [101]:
_df = (
    _cs.join(_si, how="left", on="studyId")
    .join(_vi, how="left", on="variantId")
    .select(
        "*",
        major_population_allele_freq(
            f.col("majorPopulation"),
            f.col("alleleFrequencies"),
        ).alias("majorPopulationAF"),
    )
    .select(
        "*",
        maf(f.col("majorPopulationAf")).alias("majorPopulationMAF"),
    )
)


In [ ]:
_df.count()


2622098

Save the dataset for further analysis

In [106]:
_df.write.parquet("lead-maf-vep")
